In [1]:
!pip install -q duckdb --upgrade
import sys
sys.exit(0)

sys.exit called with value 0. The interpreter will be restarted.


In [2]:
sql_folder     = './builtin/sql/'
table_base_url = 'abfss://processing@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Tables/aemo/'

In [3]:
from   deltalake     import  write_deltalake,DeltaTable
import duckdb
import re
def run_sql(list_files):
    successful_runs = 0
    for x in list_files:
        type ='default'
        try:
            file_path = f'{sql_folder}/{x}.sql'
            with open(file_path, 'r') as file:
                sql_content = file.read()

            match = re.search(r"-- materialized:\s*\((.*?)\)", sql_content)

            if match:
                materialized_content = match.group(1)
                parts = [part.strip() for part in materialized_content.split(',')]

                if len(parts) >= 3:
                    table_name = parts[0]
                    type       = parts[1]
                    mode       = parts[2]

        except FileNotFoundError:
            print(f"Error: The file '{file_path}' was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")
        if type =='default':
            try:
                    duckdb.sql(sql_content)
                    print(f"Data updated in {x}")
                    successful_runs = successful_runs +1
            except:
                    print(f"Data not updated in {x}")
        elif type =='delta':
            write_delta(sql_content,table_name, mode)
            duckdb.sql(f""" create or replace view {table_name} as select * from delta_scan('{table_base_url}{table_name}') """)
            successful_runs = successful_runs +1
        elif type =='parquet':
                try:
                        duckdb.sql(f""" copy ({sql_content}) TO '/lakehouse/default/Files/{table_name}' (FORMAT parquet, PARQUET_VERSION V2, file_size_bytes '128mb', {mode}) """)
                        print(f"Data updated in {table_name}")
                        successful_runs = successful_runs +1
                except:
                        print(f"Data not updated in {table_name}")
            
    return successful_runs
def write_delta(sql_content,tbl, mode):
        RG=8_000_000
        df = duckdb.sql(sql_content).arrow()
        if df.num_rows > 0:
            write_deltalake(table_base_url + tbl ,
            df,
            mode=mode,
            max_rows_per_file = RG ,
            max_rows_per_group = RG,
            min_rows_per_group = RG,
            engine ='pyarrow')
            if mode !='ignore':
                dt = DeltaTable(table_base_url + tbl)
                if len(dt.files()) > 90:
                    dt.optimize.compact()
                    dt.vacuum(retention_hours=0, dry_run=False, enforce_retention_duration=False)
                    dt.cleanup_metadata()
        return print(f'write table {tbl} as delta')

In [4]:
run_sql(['setup'])
run_sql(['scada_today','price_today'])
t = run_sql(['scada','price'])
print(f""" Nbr of tables changed {t}:  if any changes do backup otherwise do incremental """)
if t > 0 :
    run_sql(['duid','calendar','mstdatetime','summary_backfill'])
else :
    run_sql(['summary_incremental'])

Data updated in setup


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Data updated in scada_today


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Data updated in price_today


...

write table duid as delta
write table calendar as delta


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

write table mstdatetime as delta


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

write table summary as delta
